In [1]:
import sys
sys.path.insert(0,"..")
from fama_data.FamaData import FamaData
import pandas as pd
from statsmodels.formula.api import ols

In [2]:
fd = FamaData('../data/', end_date = '2022-09-30')

In [3]:
share_issue_delta = fd.get_net_share_issue_data()['Hi 10'] - fd.get_net_share_issue_data()['Lo 10']

shortterm_rev_delta = fd.get_shortterm_reversal_data()['Hi PRIOR'] - fd.get_shortterm_reversal_data()['Lo PRIOR']
longterm_rev_delta = fd.get_longterm_reversal_data()['Hi PRIOR'] - fd.get_longterm_reversal_data()['Lo PRIOR']

innov_delta = fd.get_10_industry_data()['HiTec'] - fd.get_10_industry_data()['NoDur'] 
growth_delta = .5*fd.get_10_industry_data()['HiTec'] + .5*fd.get_10_industry_data()['Hlth '] \
             - .5*fd.get_10_industry_data()['Utils'] - .5*fd.get_10_industry_data()['Manuf']
durable_delta = fd.get_10_industry_data()['Durbl'] - fd.get_10_industry_data()['Manuf']

service_minus_goods = 2 * fd.get_49_industry_data()['PerSv'] + 2 * fd.get_49_industry_data()['BusSv'] \
                    + fd.get_49_industry_data()['Softw'] + fd.get_49_industry_data()['Insur'] \
                    + fd.get_49_industry_data()['Fins'] + fd.get_49_industry_data()['Banks'] \
                    - fd.get_49_industry_data()['Hshld'] - fd.get_49_industry_data()['BldMt'] \
                    - fd.get_49_industry_data()['ElcEq'] - fd.get_49_industry_data()['Aero'] \
                    - fd.get_49_industry_data()['Food '] - fd.get_49_industry_data()['Oil'] \
                    - 2 * fd.get_49_industry_data()['Whlsl']

needs_minus_wants = fd.get_49_industry_data()['Food '] + fd.get_49_industry_data()['Util'] \
                    + fd.get_49_industry_data()['Hlth '] + fd.get_49_industry_data()['MedEq'] \
                    + fd.get_49_industry_data()['Drugs'] + fd.get_49_industry_data()['Insur'] \
                    - fd.get_49_industry_data()['Guns'] - fd.get_49_industry_data()['Softw'] \
                    - fd.get_49_industry_data()['Autos'] - fd.get_49_industry_data()['Whlsl'] \
                    - 2*fd.get_49_industry_data()['Hshld']


accrual_delta = fd.get_accrual_data()['Hi 10'] - fd.get_accrual_data()['Lo 10']
cashflow_price_delta = fd.get_cashflow_price_data()['Hi 10'] - fd.get_cashflow_price_data()['Lo 10']
market_beta_delta = fd.get_market_beta_data()['Hi 10'] - fd.get_market_beta_data()['Lo 10']
dividend_yield_delta = fd.get_dividend_yield_data()['Hi 10'] - fd.get_dividend_yield_data()['Lo 10']
earnings_price_delta = fd.get_earnings_price_data()['Hi 10'] - fd.get_earnings_price_data()['Lo 10']
variance_delta = fd.get_variance_data()['Hi 10'] - fd.get_variance_data()['Lo 10']


share_issue_delta.name = 'share_issue_delta'
shortterm_rev_delta.name = 'shortterm_rev_delta'
longterm_rev_delta.name = 'longterm_rev_delta'
innov_delta.name = 'innov_delta'
growth_delta.name = 'growth_delta'
durable_delta.name = 'durable_delta'
service_minus_goods.name = 'service_minus_goods'
needs_minus_wants.name = 'needs_minus_wants'
accrual_delta.name = 'accrual_delta'
cashflow_price_delta.name = 'cashflow_price_delta'
market_beta_delta.name = 'market_beta_delta'
dividend_yield_delta.name = 'dividend_yield_delta'
earnings_price_delta.name = 'earnings_price_delta'
variance_delta.name = 'variance_delta'

FileNotFoundError: [Errno 2] No such file or directory: 'data/Portfolios_Formed_on_INV.CSV'

In [ ]:
# Classic Fama-French 3-factor model components
excess_returns = fd.get_excess_return_data()['exmt']
SMB_proxy = fd.get_size_data()['Hi 10'] - fd.get_size_data()['Lo 10']
HML_proxy = fd.get_book_to_market_data()['Hi 10'] - fd.get_book_to_market_data()['Lo 10']

# Fama-French 5-factor model components
RMW_proxy = fd.get_operating_profitability_data()['Hi 10'] - fd.get_operating_profitability_data()['Lo 10']
CMA_proxy = fd.get_investment_data()['Hi 10'] - fd.get_investment_data()['Lo 10']

# And momentum
MOM_proxy = fd.get_momentum_data()['Hi PRIOR'] - fd.get_momentum_data()['Lo PRIOR']

SMB_proxy.name = 'SMB'
HML_proxy.name = 'HML'
RMW_proxy.name = 'RMW'
CMA_proxy.name = 'CMA'
MOM_proxy.name = 'MOM'

In [ ]:
growth_delta.info()

<class 'pandas.core.series.Series'>
RangeIndex: 711 entries, 0 to 710
Series name: growth_delta
Non-Null Count  Dtype  
--------------  -----  
711 non-null    float64
dtypes: float64(1)
memory usage: 5.7 KB


In [ ]:
# Our first testing portfolios
# Equal-weighted momentum portfolios
url = 'http://diether.org/prephd/10-port_mom_ew.csv'
port = pd.read_csv(url,parse_dates=['caldt'])
port = port.query('caldt >= "1963-07-31" and caldt <= "2023-02-28"')
port.info()

<class 'pandas.core.frame.DataFrame'>
Index: 711 entries, 438 to 1148
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   caldt   711 non-null    datetime64[ns]
 1   p0      711 non-null    float64       
 2   p1      711 non-null    float64       
 3   p2      711 non-null    float64       
 4   p3      711 non-null    float64       
 5   p4      711 non-null    float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 38.9 KB


In [ ]:
# Critical for concat to work properly
for x in [share_issue_delta, shortterm_rev_delta, longterm_rev_delta, 
                innov_delta, growth_delta, durable_delta, service_minus_goods, needs_minus_wants, 
                accrual_delta, cashflow_price_delta, market_beta_delta, dividend_yield_delta, 
                earnings_price_delta, variance_delta]:
    x.reset_index(drop=True, inplace=True)

In [ ]:
df = pd.concat([port, excess_returns, share_issue_delta, shortterm_rev_delta, longterm_rev_delta, 
                innov_delta, growth_delta, durable_delta, service_minus_goods, needs_minus_wants, 
                accrual_delta, cashflow_price_delta, market_beta_delta, dividend_yield_delta, 
                earnings_price_delta, variance_delta, port, SMB_proxy, HML_proxy, RMW_proxy, CMA_proxy], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   exmt                  711 non-null    float64       
 1   share_issue_delta     711 non-null    float64       
 2   shortterm_rev_delta   711 non-null    float64       
 3   longterm_rev_delta    711 non-null    float64       
 4   innov_delta           711 non-null    float64       
 5   growth_delta          711 non-null    float64       
 6   durable_delta         711 non-null    float64       
 7   service_minus_goods   711 non-null    float64       
 8   needs_minus_wants     711 non-null    float64       
 9   accrual_delta         711 non-null    float64       
 10  cashflow_price_delta  711 non-null    float64       
 11  market_beta_delta     711 non-null    float64       
 12  dividend_yield_delta  711 non-null    float64       
 13  earnings_price_delta

In [ ]:
ols('p0 ~ 1 + excess_returns + share_issue_delta + shortterm_rev_delta + longterm_rev_delta + innov_delta + growth_delta + durable_delta + service_minus_goods + needs_minus_wants + accrual_delta + cashflow_price_delta + market_beta_delta + dividend_yield_delta + earnings_price_delta + variance_delta', data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     p0   R-squared:                       0.841
Model:                            OLS   Adj. R-squared:                  0.838
Method:                 Least Squares   F-statistic:                     245.0
Date:                Thu, 13 Apr 2023   Prob (F-statistic):          1.96e-265
Time:                        23:56:51   Log-Likelihood:                -1800.0
No. Observations:                 711   AIC:                             3632.
Df Residuals:                     695   BIC:                             3705.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -0.1530      0.127     -1.208      0.227      -0.402       0.096
excess_returns          90.2910      3.681     24.530      0.000      83.064      97.518
share_issue_delta        0.1141      0.047      2.409      0.016       0.021       0.207
shortterm_rev_delta     -0.1376      0.023     -6.050      0.000      -0.182      -0.093
longterm_rev_delta      -0.0314      0.028     -1.138      0.256      -0.086       0.023
innov_delta             -0.0572      0.036     -1.572      0.116      -0.129       0.014
growth_delta            -0.0302      0.055     -0.554      0.580      -0.137       0.077
durable_delta            0.1041      0.029      3.594      0.000       0.047       0.161
service_minus_goods      0.0016      0.005      0.350      0.726      -0.007       0.011
needs_minus_wants       -0.0024      0.004     -0.598      0.550      -0.010       0.005
accrual_delta            0.1966      0.043      4.602      0.000       0.113       0.280
cashflow_price_delta     0.1451      0.061      2.386      0.017       0.026       0.264
market_beta_delta        0.1765      0.039      4.570      0.000       0.101       0.252
dividend_yield_delta     0.1897      0.033      5.693      0.000       0.124       0.255
earnings_price_delta    -0.0568      0.062     -0.912      0.362      -0.179       0.065
variance_delta           0.3028      0.030     10.057      0.000       0.244       0.362
==============================================================================
Omnibus:                      138.751   Durbin-Watson:                   2.121
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              628.629
Skew:                           0.812   Prob(JB):                    3.13e-137
Kurtosis:                       7.310   Cond. No.                     1.05e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.05e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
ols('p1 ~ 1 + excess_returns + share_issue_delta + shortterm_rev_delta + longterm_rev_delta + innov_delta + growth_delta + durable_delta + service_minus_goods + needs_minus_wants + accrual_delta + cashflow_price_delta + market_beta_delta + dividend_yield_delta + earnings_price_delta + variance_delta', data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     p1   R-squared:                       0.867
Model:                            OLS   Adj. R-squared:                  0.865
Method:                 Least Squares   F-statistic:                     303.2
Date:                Thu, 13 Apr 2023   Prob (F-statistic):          7.99e-293
Time:                        23:56:51   Log-Likelihood:                -1534.7
No. Observations:                 711   AIC:                             3101.
Df Residuals:                     695   BIC:                             3175.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.3986      0.087      4.570      0.000       0.227       0.570
excess_returns          86.9378      2.535     34.301      0.000      81.961      91.914
share_issue_delta        0.1018      0.033      3.121      0.002       0.038       0.166
shortterm_rev_delta     -0.1006      0.016     -6.420      0.000      -0.131      -0.070
longterm_rev_delta      -0.0517      0.019     -2.721      0.007      -0.089      -0.014
innov_delta             -0.0775      0.025     -3.090      0.002      -0.127      -0.028
growth_delta            -0.0974      0.038     -2.588      0.010      -0.171      -0.024
durable_delta            0.0499      0.020      2.502      0.013       0.011       0.089
service_minus_goods      0.0032      0.003      1.008      0.314      -0.003       0.009
needs_minus_wants       -0.0045      0.003     -1.618      0.106      -0.010       0.001
accrual_delta            0.1371      0.029      4.662      0.000       0.079       0.195
cashflow_price_delta     0.0612      0.042      1.463      0.144      -0.021       0.143
market_beta_delta        0.1418      0.027      5.332      0.000       0.090       0.194
dividend_yield_delta     0.0786      0.023      3.426      0.001       0.034       0.124
earnings_price_delta     0.0543      0.043      1.266      0.206      -0.030       0.139
variance_delta           0.1077      0.021      5.194      0.000       0.067       0.148
==============================================================================
Omnibus:                       82.933   Durbin-Watson:                   2.016
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              282.580
Skew:                           0.522   Prob(JB):                     4.35e-62
Kurtosis:                       5.907   Cond. No.                     1.05e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.05e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
ols('p2 ~ 1 + excess_returns + share_issue_delta + shortterm_rev_delta + longterm_rev_delta + innov_delta + growth_delta + durable_delta + service_minus_goods + needs_minus_wants + accrual_delta + cashflow_price_delta + market_beta_delta + dividend_yield_delta + earnings_price_delta + variance_delta', data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     p2   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.895
Method:                 Least Squares   F-statistic:                     405.1
Date:                Thu, 13 Apr 2023   Prob (F-statistic):               0.00
Time:                        23:56:51   Log-Likelihood:                -1327.5
No. Observations:                 711   AIC:                             2687.
Df Residuals:                     695   BIC:                             2760.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.5964      0.065      9.154      0.000       0.469       0.724
excess_returns          83.7394      1.894     44.222      0.000      80.022      87.457
share_issue_delta        0.0791      0.024      3.247      0.001       0.031       0.127
shortterm_rev_delta     -0.0531      0.012     -4.534      0.000      -0.076      -0.030
longterm_rev_delta      -0.0688      0.014     -4.844      0.000      -0.097      -0.041
innov_delta             -0.1358      0.019     -7.250      0.000      -0.173      -0.099
growth_delta            -0.0633      0.028     -2.253      0.025      -0.118      -0.008
durable_delta            0.0024      0.015      0.160      0.873      -0.027       0.032
service_minus_goods      0.0043      0.002      1.827      0.068      -0.000       0.009
needs_minus_wants       -0.0060      0.002     -2.913      0.004      -0.010      -0.002
accrual_delta            0.0718      0.022      3.268      0.001       0.029       0.115
cashflow_price_delta     0.0462      0.031      1.478      0.140      -0.015       0.108
market_beta_delta        0.1273      0.020      6.410      0.000       0.088       0.166
dividend_yield_delta     0.0187      0.017      1.093      0.275      -0.015       0.052
earnings_price_delta     0.0751      0.032      2.343      0.019       0.012       0.138
variance_delta           0.0451      0.015      2.913      0.004       0.015       0.076
==============================================================================
Omnibus:                       25.034   Durbin-Watson:                   1.867
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               62.129
Skew:                           0.032   Prob(JB):                     3.23e-14
Kurtosis:                       4.447   Cond. No.                     1.05e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.05e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
ols('p3 ~ 1 + excess_returns + share_issue_delta + shortterm_rev_delta + longterm_rev_delta + innov_delta + growth_delta + durable_delta + service_minus_goods + needs_minus_wants + accrual_delta + cashflow_price_delta + market_beta_delta + dividend_yield_delta + earnings_price_delta + variance_delta', data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     p3   R-squared:                       0.880
Model:                            OLS   Adj. R-squared:                  0.877
Method:                 Least Squares   F-statistic:                     339.9
Date:                Thu, 13 Apr 2023   Prob (F-statistic):          7.47e-308
Time:                        23:56:51   Log-Likelihood:                -1371.5
No. Observations:                 711   AIC:                             2775.
Df Residuals:                     695   BIC:                             2848.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.7998      0.069     11.539      0.000       0.664       0.936
excess_returns          83.0324      2.015     41.215      0.000      79.077      86.988
share_issue_delta        0.0875      0.026      3.375      0.001       0.037       0.138
shortterm_rev_delta     -0.0096      0.012     -0.774      0.439      -0.034       0.015
longterm_rev_delta      -0.0916      0.015     -6.065      0.000      -0.121      -0.062
innov_delta             -0.1531      0.020     -7.685      0.000      -0.192      -0.114
growth_delta            -0.0122      0.030     -0.407      0.684      -0.071       0.047
durable_delta           -0.0376      0.016     -2.369      0.018      -0.069      -0.006
service_minus_goods      0.0064      0.003      2.543      0.011       0.001       0.011
needs_minus_wants       -0.0069      0.002     -3.172      0.002      -0.011      -0.003
accrual_delta            0.0069      0.023      0.294      0.769      -0.039       0.053
cashflow_price_delta     0.0506      0.033      1.521      0.129      -0.015       0.116
market_beta_delta        0.0869      0.021      4.113      0.000       0.045       0.128
dividend_yield_delta    -0.0557      0.018     -3.055      0.002      -0.092      -0.020
earnings_price_delta     0.0799      0.034      2.344      0.019       0.013       0.147
variance_delta           0.0584      0.016      3.547      0.000       0.026       0.091
==============================================================================
Omnibus:                       37.649   Durbin-Watson:                   1.741
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               89.893
Skew:                          -0.266   Prob(JB):                     3.02e-20
Kurtosis:                       4.659   Cond. No.                     1.05e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.05e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
ols('p4 ~ 1 + excess_returns + share_issue_delta + shortterm_rev_delta + longterm_rev_delta + innov_delta + growth_delta + durable_delta + service_minus_goods + needs_minus_wants + accrual_delta + cashflow_price_delta + market_beta_delta + dividend_yield_delta + earnings_price_delta + variance_delta', data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     p4   R-squared:                       0.814
Model:                            OLS   Adj. R-squared:                  0.810
Method:                 Least Squares   F-statistic:                     202.6
Date:                Thu, 13 Apr 2023   Prob (F-statistic):          9.62e-242
Time:                        23:56:51   Log-Likelihood:                -1708.5
No. Observations:                 711   AIC:                             3449.
Df Residuals:                     695   BIC:                             3522.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                1.2162      0.111     10.923      0.000       0.998       1.435
excess_returns          87.4408      3.236     27.020      0.000      81.087      93.795
share_issue_delta        0.1224      0.042      2.941      0.003       0.041       0.204
shortterm_rev_delta      0.0508      0.020      2.541      0.011       0.012       0.090
longterm_rev_delta      -0.1271      0.024     -5.238      0.000      -0.175      -0.079
innov_delta             -0.0971      0.032     -3.033      0.003      -0.160      -0.034
growth_delta             0.0341      0.048      0.711      0.477      -0.060       0.128
durable_delta           -0.1199      0.025     -4.706      0.000      -0.170      -0.070
service_minus_goods      0.0071      0.004      1.743      0.082      -0.001       0.015
needs_minus_wants       -0.0055      0.004     -1.561      0.119      -0.012       0.001
accrual_delta           -0.0544      0.038     -1.448      0.148      -0.128       0.019
cashflow_price_delta    -0.0035      0.053     -0.066      0.948      -0.108       0.101
market_beta_delta        0.0263      0.034      0.776      0.438      -0.040       0.093
dividend_yield_delta    -0.1293      0.029     -4.416      0.000      -0.187      -0.072
earnings_price_delta     0.1091      0.055      1.992      0.047       0.002       0.217
variance_delta           0.2245      0.026      8.482      0.000       0.173       0.276
==============================================================================
Omnibus:                       79.739   Durbin-Watson:                   1.738
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              545.613
Skew:                           0.184   Prob(JB):                    3.32e-119
Kurtosis:                       7.276   Cond. No.                     1.05e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.05e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
ols('p4 - p0 ~ 1 + excess_returns + share_issue_delta + shortterm_rev_delta + longterm_rev_delta + innov_delta + growth_delta + durable_delta + service_minus_goods + needs_minus_wants + accrual_delta + cashflow_price_delta + market_beta_delta + dividend_yield_delta + earnings_price_delta + variance_delta', data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     p4   R-squared:                       0.814
Model:                            OLS   Adj. R-squared:                  0.810
Method:                 Least Squares   F-statistic:                     202.6
Date:                Thu, 13 Apr 2023   Prob (F-statistic):          9.62e-242
Time:                        23:56:51   Log-Likelihood:                -1708.5
No. Observations:                 711   AIC:                             3449.
Df Residuals:                     695   BIC:                             3522.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                1.2162      0.111     10.923      0.000       0.998       1.435
excess_returns          87.4408      3.236     27.020      0.000      81.087      93.795
share_issue_delta        0.1224      0.042      2.941      0.003       0.041       0.204
shortterm_rev_delta      0.0508      0.020      2.541      0.011       0.012       0.090
longterm_rev_delta      -0.1271      0.024     -5.238      0.000      -0.175      -0.079
innov_delta             -0.0971      0.032     -3.033      0.003      -0.160      -0.034
growth_delta             0.0341      0.048      0.711      0.477      -0.060       0.128
durable_delta           -0.1199      0.025     -4.706      0.000      -0.170      -0.070
service_minus_goods      0.0071      0.004      1.743      0.082      -0.001       0.015
needs_minus_wants       -0.0055      0.004     -1.561      0.119      -0.012       0.001
accrual_delta           -0.0544      0.038     -1.448      0.148      -0.128       0.019
cashflow_price_delta    -0.0035      0.053     -0.066      0.948      -0.108       0.101
market_beta_delta        0.0263      0.034      0.776      0.438      -0.040       0.093
dividend_yield_delta    -0.1293      0.029     -4.416      0.000      -0.187      -0.072
earnings_price_delta     0.1091      0.055      1.992      0.047       0.002       0.217
variance_delta           0.2245      0.026      8.482      0.000       0.173       0.276
==============================================================================
Omnibus:                       79.739   Durbin-Watson:                   1.738
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              545.613
Skew:                           0.184   Prob(JB):                    3.32e-119
Kurtosis:                       7.276   Cond. No.                     1.05e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.05e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
ols('p1 ~ 1 + excess_returns + SMB + HML + RMW + CMA + share_issue_delta + shortterm_rev_delta + longterm_rev_delta + innov_delta + growth_delta + durable_delta + service_minus_goods + needs_minus_wants + accrual_delta + cashflow_price_delta + market_beta_delta + dividend_yield_delta + earnings_price_delta + variance_delta', data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     p1   R-squared:                       0.895
Model:                            OLS   Adj. R-squared:                  0.892
Method:                 Least Squares   F-statistic:                     309.4
Date:                Thu, 13 Apr 2023   Prob (F-statistic):          1.68e-322
Time:                        23:56:51   Log-Likelihood:                -1452.5
No. Observations:                 711   AIC:                             2945.
Df Residuals:                     691   BIC:                             3036.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.2630      0.079      3.320      0.001       0.107       0.418
excess_returns          94.4593      2.410     39.198      0.000      89.728      99.191
SMB                     -0.2815      0.024    -11.833      0.000      -0.328      -0.235
HML                      0.1119      0.028      4.067      0.000       0.058       0.166
RMW                      0.0088      0.027      0.331      0.741      -0.043       0.061
CMA                      0.1346      0.029      4.600      0.000       0.077       0.192
share_issue_delta        0.0095      0.032      0.299      0.765      -0.053       0.072
shortterm_rev_delta     -0.0923      0.014     -6.526      0.000      -0.120      -0.065
longterm_rev_delta       0.0052      0.020      0.266      0.790      -0.033       0.044
innov_delta             -0.0457      0.023     -1.951      0.051      -0.092       0.000
growth_delta            -0.0782      0.034     -2.278      0.023      -0.146      -0.011
durable_delta            0.0635      0.018      3.538      0.000       0.028       0.099
service_minus_goods      0.0015      0.003      0.541      0.589      -0.004       0.007
needs_minus_wants       -0.0018      0.002     -0.722      0.470      -0.007       0.003
accrual_delta            0.1026      0.027      3.805      0.000       0.050       0.155
cashflow_price_delta     0.0338      0.038      0.889      0.374      -0.041       0.108
market_beta_delta        0.0906      0.024      3.752      0.000       0.043       0.138
dividend_yield_delta     0.0712      0.021      3.430      0.001       0.030       0.112
earnings_price_delta    -0.0124      0.039     -0.314      0.754      -0.090       0.065
variance_delta           0.0084      0.021      0.395      0.693      -0.033       0.050
==============================================================================
Omnibus:                      125.822   Durbin-Watson:                   1.900
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              989.358
Skew:                           0.542   Prob(JB):                    1.46e-215
Kurtosis:                       8.676   Cond. No.                     1.12e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.12e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
ols('p3 ~ 1 + excess_returns + HML + SMB', data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     p3   R-squared:                       0.934
Model:                            OLS   Adj. R-squared:                  0.934
Method:                 Least Squares   F-statistic:                     3340.
Date:                Thu, 13 Apr 2023   Prob (F-statistic):               0.00
Time:                        23:56:51   Log-Likelihood:                -1158.6
No. Observations:                 711   AIC:                             2325.
Df Residuals:                     707   BIC:                             2344.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.6774      0.047     14.451      0.000       0.585       0.769
excess_returns    89.5012      1.049     85.299      0.000      87.441      91.561
HML                0.0777      0.011      7.307      0.000       0.057       0.099
SMB               -0.3311      0.011    -30.587      0.000      -0.352      -0.310
==============================================================================
Omnibus:                       80.999   Durbin-Watson:                   1.728
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              231.479
Skew:                          -0.565   Prob(JB):                     5.43e-51
Kurtosis:                       5.557   Cond. No.                         130.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""